In [1]:
import tensorflow as tf
import numpy as np
from moviepy.editor import *
from pydub import AudioSegment

audio = AudioSegment.from_file('/home/andrea/Scrivania/test_video.mp4')
#audio = video.audio

In [2]:
dataset = np.load("/home/andrea/Scrivania/MIVIA_ROAD_DB1/dataset.npy")
std_dev = np.std(dataset)
mean = np.mean(dataset)
maximum = np.max(dataset)
min = np.min(dataset)
scale = max(maximum, abs(min))

In [3]:
def normalize(array, std_dev, mean, scale):
    array = array/scale
    return array-mean/std_dev

In [4]:
samples = audio.get_array_of_samples()
samples = samples[:264256]

In [6]:

step = 16516
def split_in_windows(sample):
    windows = np.zeros((16,16516))
    for j in range(16):
        windows[j,:] = sample[j*step:(j+1)*step]
    return windows
windows = split_in_windows(samples)

In [7]:
norm_windows = normalize(windows,std_dev,mean,scale)

In [8]:
from tensorflow import keras
model = keras.models.load_model("audio_model_bidirectional_LSTM.h5")

In [26]:
test = np.zeros((16,16,16516))
test2 = np.zeros((16,16,16516))
for i,w in enumerate(windows):
    for j in range(16):
        j2 = (i+j) % 16
        test2[j,j2,:] = w

In [21]:
test[0,:,:] == windows

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [22]:
aug = windows[np.newaxis,:]

In [24]:
model.predict_classes(test)

array([[0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0]], dtype=int32)

In [42]:
arr = test2[4].flatten()
int_array = arr.astype(np.int16)

In [43]:
accident = AudioSegment(int_array.tobytes(),
                        frame_rate=audio.frame_rate,
                      sample_width=audio.frame_width,
                      channels=1)
accident.export("/home/andrea/Scrivania/accident.wav", format="wav")

<_io.BufferedRandom name='/home/andrea/Scrivania/accident.wav'>